In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from data_loader import (
    load_etfs_ibovespa,
    load_xfix11,
    load_ifix,
    load_selic_data,
    combine_fii_series
)

from scripts.lft_model import simulate_lft
from scripts.portfolio_simulator import (
    calculate_log_returns,
    simulate_by_period,
    simulate_6040_portfolio
)

from scripts.benchmarking import compare_portfolios

etfs = load_etfs_ibovespa()
xfix = load_xfix11()
ifix = load_ifix()
selic_df = load_selic_data()
lft_series = simulate_lft(selic_df)
xfix_combined = combine_fii_series(ifix, xfix)

full_data = pd.concat([etfs, xfix_combined, lft_series], axis=1).dropna()
returns_df = calculate_log_returns(full_data)

periods = [10, 5, 3]
weights_dict = simulate_by_period(full_data, periods)

portfolio_returns_dict = {}

for label, weights in weights_dict.items():
    efficient = returns_df.dot(weights)
    portfolio_returns_dict[f"Efficient_{label}"] = efficient
    portfolio_6040 = simulate_6040_portfolio(weights, returns_df, returns_df["LFT"])
    portfolio_returns_dict[f"60_40_{label}"] = portfolio_6040

ibov_returns = returns_df["IBOVESPA"]
benchmark_df = compare_portfolios(portfolio_returns_dict, ibov_returns)

benchmark_df.style.format({
    "Annual Return": "{:.2%}",
    "Volatility": "{:.2%}",
    "Sharpe": "{:.2%}",
    "Tracking Error": "{:.2%}",
    "Excess Return": "{:.2%}"
})

plt.figure(figsize=(12,6))
for name, returns in portfolio_returns_dict.items():
    cumulative = (1 + returns).cumprod()
    plt.plot(cumulative, label=name)

plt.plot((1 + ibov_returns).cumprod(), label="IBOVESPA", linestyle="--", color="black")
plt.title("Cumulative Returns Comparison")
plt.legend()
plt.grid(True)
plt.show()

benchmark_df.to_csv("dashboard/portfolio_metrics.csv", index=False)
returns_df.to_csv("dashboard/full_returns.csv")

print("Full data shape:", full_data.shape)
print("Returns shape:", returns_df.shape)
print("Weights dict:", weights_dict)
print("Portfolio keys:", portfolio_returns_dict.keys())
print("Benchmark head:")
print(benchmark_df.head())